In [5]:
!pip install transformers

### imports

In [2]:
import json
from transformers import pipeline

import numpy as np

from pprint import pprint
from tensorflow.keras.optimizers import Adam

### Use transformers get a model for predicting MGT

In [ ]:
pipe = pipeline(model="facebook/bart-large-mnli")

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

### load data from jsonl

In [ ]:
def load_data(file_path):
  data_list = []
  with open(file_path, 'r') as file:
    for line in file:
        # load each line as a JSON object
        data = json.loads(line.strip())
        data_list.append(data)
  return data_list


In [6]:
file_path = 'drive/MyDrive/models/subtaskA_dev_monolingual.jsonl'

In [ ]:
data_loaded = load_data(file_path)

### get a random sample from data

In [ ]:
test_data = data_loaded[100]['text']
print(test_data)
print("label is: ", data_loaded[100]['label'])

If you can't get your Samsung Galaxy S3 connected to your computer using its default connection method -- MTP or PTP -- you'll need to use an alternative way.  This guide will show you how to do that by enabling ADB debugging mode instead.   Note that this process may void... If you're still having trouble getting your S3 to work properly after following these steps, try contacting Samsung directly via their support page here. You can also visit our troubleshooting section if you'd like more help fixing other issues related to your Android smartphone. Restart both devices before continuing so they are ready to be reconnected.    The first thing we recommend doing is trying out another USB cord and/or USB port since many users have reported success simply changing cables/ports while attempting to fix problems between phones/computers.     To check which ports are currently being used,...   -  Plug one end of the new USB cord into your computer's charging port..
-  Plug the other end of 

### Test using pipeline

In [ ]:
# test this using the pipeline
result = pipe(test_data,
              candidate_labels=["Written by humans", "Written by AI"],)

In [ ]:
print(type(test_data))

<class 'str'>


In [ ]:
pprint(result)

{'labels': ['Written by humans', 'Written by AI'],
 'scores': [0.5420032143592834, 0.45799681544303894],
 'sequence': "If you can't get your Samsung Galaxy S3 connected to your "
             'computer using its default connection method -- MTP or PTP -- '
             "you'll need to use an alternative way.  This guide will show you "
             'how to do that by enabling ADB debugging mode instead.   Note '
             "that this process may void... If you're still having trouble "
             'getting your S3 to work properly after following these steps, '
             'try contacting Samsung directly via their support page here. You '
             "can also visit our troubleshooting section if you'd like more "
             'help fixing other issues related to your Android smartphone. '
             'Restart both devices before continuing so they are ready to be '
             'reconnected.    The first thing we recommend doing is trying out '
             'another USB cord an

In [ ]:
def prepare_test_data():
  texts = []
  labels = []
  for data in data_loaded:
    texts.append(data['text'])
    labels.append(data['label'])
  return texts,labels


In [ ]:
x_test, y_test = prepare_test_data()

In [ ]:
print(len(x_test))

5000


In [ ]:
x_test_sample = x_test[:3]
y_test_sample = y_test[:3]

In [ ]:
results = pipe(x_test_sample,
              candidate_labels=["Written by humans", "Written by AI"],)

In [ ]:
pprint(results)

In [ ]:
# verify results
correct = 0
for i in range(len(results)):
    # take the index of the label with the highet score from scores
    if results[i]['scores'].index(max(results[i]['scores'])) == y_test[i]:
      correct +=1
print("Accuracy is : ", correct / len(results))

Accuracy is :  0.0


# Zero-shot classification


In [ ]:
classifier = pipeline("zero-shot-classification")
classifier(
    x_test_sample,
    candidate_labels=["human", "AI"],
)

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'sequence': "Giving gifts should always be enjoyable.  However, it may become stressful when trying to find that perfect present.   This wikiHow will help you figure out exactly what you'd love to receive this year!   If you're having trouble deciding between two different items (or more), try making lists of advantages/disadvantages so you'll know which one would make the best choice.    Make sure it's appropriate - some people don't appreciate receiving certain types of presents from their friends and/or family members.     Don't forget to include any special requests you've already made before now; these might have been forgotten by others who haven't seen them yet.        Write down all... Continue reading →\n\nIf you can't think of anything specific right away but still feel like getting something nice,  consider giving yourself a treat instead!  Here are just a few suggestions:  Buying yourself flowers Gifting yourself tickets to see your favorite band Take time off work Treat 

# Fine-tuning: using BERT pre-trained model!

In [ ]:
x_test, y_test = prepare_test_data()

In [ ]:
print(len(x_test))

5000


### Split data into train and test

In [ ]:
# split the data into train and test set
X_train, X_test, y_train, y_test = train_test_split(x_test, y_test, test_size=0.2, random_state=42)

In [ ]:
# print the shapes of the train and test set
print(len(X_train), len(y_train), len(X_test), len(y_test))

4000 4000 1000 1000


In [ ]:
# load the pre-trained BERt tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# convert labels to numpy array
# the labels are already a list of 0 and 1s, so we can just convert that directly to a NumPy array without tokenization!
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
# Tokenize and pad the input sequences
X_train_tokens = tokenizer(X_train, padding=True, truncation=True, max_length=128, return_tensors="tf")
X_test_tokens = tokenizer(X_test, padding=True, truncation=True, max_length=128, return_tensors="tf")


In [ ]:
print(X_train_tokens[0])

In [ ]:
# prepare the model for training
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train_tokens['input_ids'], y_train, epochs=2, batch_size=32, validation_data=(X_test_tokens['input_ids'], y_test))

Epoch 1/2
  3/125 [..............................] - ETA: 1:45:18 - loss: 5.4143 - accuracy: 0.4688

In [ ]:
# evaluate the model on the test set:
scores = model.evaluate(X_test_tokens['input_ids'], y_test)
print("Accuracy:", scores[1])

### version 2

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
data_loaded[0]['text']

"Giving gifts should always be enjoyable.  However, it may become stressful when trying to find that perfect present.   This wikiHow will help you figure out exactly what you'd love to receive this year!   If you're having trouble deciding between two different items (or more), try making lists of advantages/disadvantages so you'll know which one would make the best choice.    Make sure it's appropriate - some people don't appreciate receiving certain types of presents from their friends and/or family members.     Don't forget to include any special requests you've already made before now; these might have been forgotten by others who haven't seen them yet.        Write down all... Continue reading →\n\nIf you can't think of anything specific right away but still feel like getting something nice,  consider giving yourself a treat instead!  Here are just a few suggestions:  Buying yourself flowers Gifting yourself tickets to see your favorite band Take time off work Treat yourself to lu

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenized_data = tokenizer(X_train, return_tensors="np", padding=True, truncation=True, max_length=128)

In [ ]:
tokenized_data = dict(tokenized_data)

In [ ]:
# Load and compile our model
# for subtask A
id2label = {0 : "human" , 1: "machine"}
label2id = {"human": 0, "machine": 1}
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased")

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.compile(Adam(3e-5))  # No loss argument!
# Hugging Face models automatically choose a loss that is appropriate for their task

In [ ]:
model.fit(tokenized_data, y_train)

 13/125 [==>...........................] - ETA: 1:45:07 - loss: 0.7109

## Version 3

In [7]:
!pip install Datasets

In [222]:
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import DataCollatorWithPadding
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import BertModel, BertTokenizer, TFBertForSequenceClassification, TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam
import datasets

In [223]:
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()

In [224]:
def preprocess_function(examples, tokenizer, max_length=128):
    return tokenizer(examples["text"],
                     truncation=True,
                     padding='max_length',
                     max_length=max_length,
                     return_tensors="np")

In [225]:
def get_data(train_path):
   train_df = pd.read_json(train_path, lines=True)
   train_df, test_df = train_test_split(train_df, test_size=0.2, stratify=train_df['label'], random_state=42)
   return train_df, test_df


In [226]:
train_df, test_df = get_data(file_path)

In [227]:
labels = np.asarray(train_df['label'])

In [228]:
id2label = {0: "human", 1: "machine"}
label2id = {"human": 0, "machine": 1 }

In [229]:
 # pandas dataframe to huggingface Dataset
train_dataset_ours = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [230]:
print(train_dataset_ours)

Dataset({
    features: ['text', 'label', 'model', 'source', 'id', '__index_level_0__'],
    num_rows: 4000
})


In [231]:
# convert to train_dataset_ours to <class 'datasets.dataset_dict.DatasetDict'>
new_dataset = datasets.DatasetDict({'train': train_dataset_ours, 'test': test_dataset})

In [232]:
print(new_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'model', 'source', 'id', '__index_level_0__'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['text', 'label', 'model', 'source', 'id', '__index_level_0__'],
        num_rows: 1000
    })
})


In [233]:
new_dataset["train"][0]

{'text': 'They have been accused of being greedy and manipulative businessmen who use any means necessary to get what they want, including bribery or extortion.  They also own many companies in different industries which makes them very powerful. The Koch family has donated millions towards conservative causes such as anti-abortion groups and climate change denial organizations. Their donations were made through foundations set up by David H. Koch\'s wife, Anne Scaife. Some conservatives believe this money was used to influence politicians into making decisions favorable toward the Kochs interests. They have been called "the most dangerous Americans" because some conservatives think they control too much power over our government. They have been accused of being  greedy and manipulative businessman who use any means necessary .to get what they want ,including bribery or extorsion .\nthey also own many companiess in diffrent industryes whitch make em very powereful',
 'label': 1,
 'mode

In [234]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [210]:
tokenizer("Hello, this is a sentence!", "And this sentence goes with it.")

{'input_ids': [101, 8667, 117, 1142, 1110, 170, 5650, 106, 102, 1262, 1142, 5650, 2947, 1114, 1122, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [235]:
def preprocess_function(examples):
    # Keys of the returned dictionary will be added to the dataset as columns
    return tokenizer(examples["text"],truncation=True, padding='max_length', max_length=128)

In [236]:
pre_tokenizer_columns = set(new_dataset["train"].features)
encoded_dataset = new_dataset.map(preprocess_function, batched=True)
tokenizer_columns = list(set(encoded_dataset["train"].features) - pre_tokenizer_columns)
print("Columns added by tokenizer:", tokenizer_columns)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Columns added by tokenizer: ['input_ids', 'attention_mask', 'token_type_ids']


In [263]:
encoded_dataset["train"].features["label"]

Value(dtype='int64', id=None)

In [238]:
tf_train_dataset = model.prepare_tf_dataset(encoded_dataset["train"],
                                      batch_size=2,
                                      shuffle=True,
                                      tokenizer=tokenizer)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [239]:
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label2id), id2label=id2label, label2id=label2id)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [240]:
model.compile(optimizer=Adam(3e-5))  # No loss argument!

In [241]:
model.fit(tf_train_dataset)

   3/2000 [..............................] - ETA: 4:19:45 - loss: 0.6645

KeyboardInterrupt: ignored

### 2 directions: -> Change in BERT model or Change in Tokenizer model.

In [ ]:
https://stackoverflow.com/questions/64156202/add-dense-layer-on-top-of-huggingface-bert-model

In [268]:
from tensorflow import keras
from transformers import AutoTokenizer, TFAutoModel
class CustomBERTModel(keras.Model):
    def __init__(self):
          super(CustomBERTModel, self).__init__()
          self.bert = TFAutoModel.from_pretrained("bert-base-uncased")
          ### New layers:
          self.linear1 = keras.layers.Dense(256)
          self.linear2 = keras.layers.Dense(2) ## 2 is the number of classes in this example

    def call(self, inputs, training=False):
          # call expects only one positional argument, so you have to pass in a tuple and unpack. The next parameter is a special reserved training parameter.
          ids, mask = inputs
          sequence_output = self.bert(ids, mask, training=training).last_hidden_state

          # sequence_output has the following shape: (batch_size, sequence_length, 768)
          linear1_output = self.linear1(sequence_output[:,0,:]) ## extract the 1st token's embeddings

          linear2_output = self.linear2(linear1_output)

          return linear2_output


In [269]:
model = CustomBERTModel()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture

In [272]:
epochs = 2
# train the model by using GradientTape
optimizer = keras.optimizers.Adam(learning_rate=5e-5)
for epoch in range(epochs):
    print(f"\nStart of Training Epoch {epoch}")
    for step, batch in enumerate(tf_train_dataset):
        # print(step)
        # print(batch)
        ids = batch[0]['input_ids']
        mask = batch[0]['attention_mask']
        y = batch[1]
        with tf.GradientTape() as tape:
            logits = model((ids, mask), training=True)
            loss_value = SparseCategoricalCrossentropy(from_logits=True)(y, logits)
            print(f"Loss at step {step}: {loss_value}")
        grads = tape.gradient(loss_value, model.trainable_weights)
        # Filter trainable weights that have gradients
        trainable_vars = [var for var, grad in zip(model.trainable_weights, grads) if grad is not None]

        optimizer.apply_gradients(zip(grads, model.trainable_weights))




Start of Training Epoch 0
0
({'input_ids': <tf.Tensor: shape=(2, 128), dtype=int64, numpy=
array([[  101, 13030,  3171,  1324,  3177,  1324,   113,  3886,   131,
          571, 26259, 28483, 15389,   573, 15389,   117,  1145,  7278,
         1112, 13030,  3171,  1324,   141,  9663,  1324,   132,  1255,
         1347,  1479,   117,  2424,   114,  1110,  1126,  7239,  1709,
         1591,  1150,  1971,  2399,  1107,  1103,  3398,  5096,  1453,
         1526, 21935,  3556, 11907, 16525, 10732,   119,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,

1
({'input_ids': <tf.Tensor: shape=(2, 128), dtype=int64, numpy=
array([[  101,  3278,  1103,  1314,  4397,  2012,  3268,  1138,  1151,
         1189,  1106,  6058,  1207,  2000,  7107,  7951,  1106,  1103,
         2025,  1105,  1936, 24647,  1104,  2012, 22555,   119,  1130,
         1142,  2025,  1195,  1329, 15278, 12880,  2069,   120,   157,
         9637,  9664,  4321, 20629,  4412,  5989,  4351,  1111, 11432,
         1105,  3622,  1104,  2607,  1107,  1103,  1449,  1104,  1413,
        11462,  1116,  2166,  1106,   170,  2012,  8386,   119,   138,
         1413, 11462,  1110,   170,  2632,  1137,   170,  4742,  8981,
         2672,  1107,  1126,  3077,   117,  1134,  1122,  1110,  1936,
         1106, 11552,  1118,   170,  1957,  6165,  1104,  4351,  1359,
         1113, 23317, 26299,  1105,  1137,  9800,  6289, 11303,   119,
          107,  1109,  2800, 11462, 18684,  5796,  1105, 18534,  1279,
         1457, 11745,  5668, 12504,   107,   113,   149,  9919,  8055,
          11

2
({'input_ids': <tf.Tensor: shape=(2, 128), dtype=int64, numpy=
array([[  101,  1247,  1132,  1672,  7997,  1107,  2131, 10104,  1134,
         4021,  1106,  4137,  1142,   119,  1109,  2440,  2749,  1115,
          146,  7572,  2059,  1110,  2276,  1112,  2131,  1110,  1142,
          131,  1875,  1144,  1640,  1107,   170,  2305,  2378, 18166,
         1105,  1189,  1140,   112,  1540,  2008,   112,   119,  1188,
         2171,  1194,  3766,  5694,  1113,  1103,  2771,   119,  1332,
         3766,  1452,  1119,  1261,  1103, 11850,  1104,  1103,  1362,
         1852,  1140,   117,  1177,  1115,  2256,  1150,  6616,  1107,
         1140,  1281,  1204,  2939,  1133,  1138, 14822,  1297,   113,
         1287,   122,   131,  1853,   117,  1287,   124,   131,  1479,
          114,   119, 18166, 11053,  1111,  1234,  1106,  2939,  1105,
         3843,  1234,  1121,  1217,  1107,  2398,  1114,  3766,   119,
         3766,   112,  2378,   112,  1140,  1118,  3558,   170,  1236,
         110

KeyboardInterrupt: ignored